In [1]:
%%time
import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE,RandomOverSampler
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from itertools import combinations 
from mlxtend.classifier import StackingClassifier
from sklearn import model_selection
import copy 

Using TensorFlow backend.
/home/sf/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sf/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sf/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sf/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtype

CPU times: user 2.28 s, sys: 670 ms, total: 2.95 s
Wall time: 2.14 s


In [2]:
df=pd.read_csv('60s_window_wrist_chest.csv',index_col=0)


In [3]:
features=df.columns.tolist()
features

removed = ['label']
for rem in removed:
    features.remove(rem)

features_with_sub=[]
features_with_sub[:]=features
removed = ['subject']
for rem in removed:
    features.remove(rem)

feature=features
print(len(feature))
len(features_with_sub)

sm = SMOTE(random_state=2)
X, y= sm.fit_sample(df[features_with_sub], df['label'])
df_new=pd.concat([pd.DataFrame(X,columns=features_with_sub),pd.DataFrame(y,columns=['label'])],axis=1)
df_new

for i in range (len(list(df_new['subject']))):
    df_new['subject'][i] = min([2,3,4,5,6,7,8,9,10,11,13,14,15,16,17], key=lambda x:abs(x-df_new['subject'][i]))
df_new['subject']=df_new['subject'].astype(int)

p_d=pd.read_csv('personal_detail.csv',index_col=0)

df_new_1=df_new.merge(p_d,on='subject')
df_new_1

70


/home/sf/.local/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,net_acc_mean,net_acc_std,net_acc_min,net_acc_max,ACC_x_mean,ACC_x_std,ACC_x_min,ACC_x_max,ACC_y_mean,ACC_y_std,...,label,age,height,weight,gender_ female,coffee_today_YES,sport_today_YES,smoker_YES,feel_ill_today_YES,bmi
0,0.025961,0.013811,0.000000,0.087383,0.023431,0.017769,-0.037843,0.087383,0.000016,1.222594e-05,...,0,27,175,80,0,0,0,0,0,1
1,0.027640,0.010597,0.002752,0.054356,0.027640,0.010597,0.002752,0.054356,0.000019,7.290999e-06,...,0,27,175,80,0,0,0,0,0,1
2,0.028389,0.006937,0.000000,0.066053,0.028378,0.006985,-0.002752,0.066053,0.000020,4.805734e-06,...,0,27,175,80,0,0,0,0,0,1
3,0.033268,0.007670,0.000000,0.074998,0.032960,0.008904,-0.030962,0.074998,0.000023,6.126303e-06,...,0,27,175,80,0,0,0,0,0,1
4,0.037021,0.001284,0.027522,0.043347,0.037021,0.001284,0.027522,0.043347,0.000025,8.837530e-07,...,0,27,175,80,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1243,0.029484,0.002074,0.020918,0.053804,-0.004624,0.002074,-0.015439,0.017447,-0.000003,1.427082e-06,...,1,29,165,55,1,0,0,0,0,0
1244,0.032744,0.000516,0.029211,0.034857,-0.029334,0.000516,-0.031478,-0.025832,-0.000020,3.552867e-07,...,2,29,165,55,1,0,0,0,0,0
1245,0.030006,0.007051,0.002966,0.070357,-0.027424,0.007051,-0.067796,-0.000404,-0.000019,4.851210e-06,...,2,29,165,55,1,0,0,0,0,0
1246,0.031250,0.001534,0.027188,0.038575,-0.031250,0.001534,-0.038575,-0.027188,-0.000022,1.055452e-06,...,2,29,165,55,1,0,0,0,0,0


In [4]:
sel_fea = ['EDA_tonic_mean','EDA_smna_mean','EDA_tonic_min','EDA_phasic_mean','TEMP_std','BVP_peak_freq','smoker_YES','ACC_y_min','ACC_x_mean','weight','gender_ female','c_Temp_max','ACC_x_max','TEMP_mean',
          'c_ACC_y_std','net_acc_max','Resp_std']

In [5]:
# train=df_new_1[df_new_1['subject']<=9]
# test=df_new_1[df_new_1['subject']>9]

train= df_new_1.loc[df_new_1.subject.isin([4, 5, 6, 7, 8, 9, 11, 17])]
test= df_new_1.loc[df_new_1.subject.isin([2, 3, 10, 13, 14, 15, 16])]

#EXTRA Trees

In [6]:
scaler = Normalizer()
scaled_data_train = scaler.fit_transform(train[sel_fea])
scaled_data_test = scaler.transform(test[sel_fea])


clf = ExtraTreesClassifier(n_estimators=100,n_jobs=10,random_state=61)
clf.fit(scaled_data_train,train['label'])
y_pred=clf.predict(scaled_data_test)
print (classification_report(test['label'],y_pred))

              precision    recall  f1-score   support

           0       0.88      0.71      0.78       147
           1       0.84      0.93      0.88       161
           2       0.68      0.74      0.71       146
           3       0.79      0.77      0.78       149

    accuracy                           0.79       603
   macro avg       0.80      0.79      0.79       603
weighted avg       0.80      0.79      0.79       603



In [7]:
# test_sub=[10,11,13,14,15,16,17]
# for i in test_sub:
#     print(i)
#     train=df_new_1[df_new_1['subject']<=9]
#     test=df_new_1[df_new_1['subject']==i]

#     scaler = Normalizer()
#     scaled_data_train = scaler.fit_transform(train[sel_fea])
#     scaled_data_test = scaler.transform(test[sel_fea])
    
#     #et = ExtraTreesClassifier(n_estimators=100,n_jobs=10,random_state=61)
#     clf.fit(scaled_data_train,train['label'])
#     y_pred=clf.predict(scaled_data_test)
#     print(classification_report(test['label'],y_pred))

#RANDOM-FORREST

In [10]:
#random state = 17
scaler = Normalizer()
scaled_data_train = scaler.fit_transform(train[sel_fea])
scaled_data_test = scaler.transform(test[sel_fea])


from sklearn.ensemble import RandomForestClassifier

for i in range (101):
    clf=RandomForestClassifier(n_estimators=50,random_state=i)
    clf.fit(scaled_data_train,train['label'])
    y_pred=clf.predict(scaled_data_test)
#     print (classification_report(test['label'],y_pred))
    
    rpt = classification_report(test['label'],y_pred,output_dict=True)['accuracy']
    
    if rpt>=0.79:
        print (i)
        print (classification_report(test['label'],y_pred))

17
              precision    recall  f1-score   support

           0       0.87      0.70      0.78       145
           1       0.90      0.94      0.92       151
           2       0.69      0.74      0.71       142
           3       0.72      0.77      0.74       145

    accuracy                           0.79       583
   macro avg       0.79      0.79      0.79       583
weighted avg       0.80      0.79      0.79       583



In [12]:
# test_sub=[10,11,13,14,15,16,17]
# for i in test_sub:
#     print(i)
#     train=df_new_1[df_new_1['subject']<=9]
#     test=df_new_1[df_new_1['subject']==i]

#     scaler = Normalizer()
#     scaled_data_train = scaler.fit_transform(train[sel_fea])
#     scaled_data_test = scaler.transform(test[sel_fea])
    
#     #et = ExtraTreesClassifier(n_estimators=100,n_jobs=10,random_state=61)
#     clf.fit(scaled_data_train,train['label'])
#     y_pred=clf.predict(scaled_data_test)
#     print(classification_report(test['label'],y_pred))

#Decision-Tree

In [18]:
#random state - 94
scaler = Normalizer()
scaled_data_train = scaler.fit_transform(train[sel_fea])
scaled_data_test = scaler.transform(test[sel_fea])


from sklearn.tree import DecisionTreeClassifier

for i in range (101):
    clf = DecisionTreeClassifier(random_state=i)
    clf.fit(scaled_data_train,train['label'])
    y_pred=clf.predict(scaled_data_test)
#     print (classification_report(test['label'],y_pred))

    rpt = classification_report(test['label'],y_pred,output_dict=True)['accuracy']

    if rpt>=0.68:
        print (i)
        print (classification_report(test['label'],y_pred))

94
              precision    recall  f1-score   support

           0       0.55      0.57      0.56       145
           1       0.85      0.93      0.89       151
           2       0.70      0.66      0.68       142
           3       0.62      0.57      0.59       145

    accuracy                           0.68       583
   macro avg       0.68      0.68      0.68       583
weighted avg       0.68      0.68      0.68       583



In [19]:
# test_sub=[10,11,13,14,15,16,17]
# for i in test_sub:
#     print(i)
#     train=df_new_1[df_new_1['subject']<=9]
#     test=df_new_1[df_new_1['subject']==i]

#     scaler = Normalizer()
#     scaled_data_train = scaler.fit_transform(train[sel_fea])
#     scaled_data_test = scaler.transform(test[sel_fea])
    
#     #et = ExtraTreesClassifier(n_estimators=100,n_jobs=10,random_state=61)
#     clf.fit(scaled_data_train,train['label'])
#     y_pred=clf.predict(scaled_data_test)
#     print(classification_report(test['label'],y_pred))

#Gaussian

In [ ]:
scaler = Normalizer()
scaled_data_train = scaler.fit_transform(train[sel_fea])
scaled_data_test = scaler.transform(test[sel_fea])


from sklearn.naive_bayes import GaussianNB 
clf = GaussianNB()
clf.fit(scaled_data_train,train['label'])
y_pred=clf.predict(scaled_data_test)
print (classification_report(test['label'],y_pred))

In [ ]:
test_sub=[10,11,13,14,15,16,17]
for i in test_sub:
    print(i)
    train=df_new_1[df_new_1['subject']<=9]
    test=df_new_1[df_new_1['subject']==i]

    scaler = Normalizer()
    scaled_data_train = scaler.fit_transform(train[sel_fea])
    scaled_data_test = scaler.transform(test[sel_fea])
    
    #et = ExtraTreesClassifier(n_estimators=100,n_jobs=10,random_state=61)
    clf.fit(scaled_data_train,train['label'])
    y_pred=clf.predict(scaled_data_test)
    print(classification_report(test['label'],y_pred))

#XG-Boost

In [8]:
scaler = Normalizer()
scaled_data_train = scaler.fit_transform(train[sel_fea])
scaled_data_test = scaler.transform(test[sel_fea])


from xgboost import XGBClassifier
import xgboost as xgb

for i in range (101):
    clf = xgb.XGBClassifier(random_state=i)
    clf.fit(scaled_data_train,train['label'])
    y_pred=clf.predict(scaled_data_test)
#     print (classification_report(test['label'],y_pred))
    rpt = classification_report(test['label'],y_pred,output_dict=True)['accuracy']
    
    if rpt>=0.77:
        print (i)
        print (classification_report(test['label'],y_pred))

0
              precision    recall  f1-score   support

           0       0.87      0.66      0.75       145
           1       0.83      0.93      0.88       151
           2       0.67      0.76      0.71       142
           3       0.74      0.72      0.73       145

    accuracy                           0.77       583
   macro avg       0.78      0.77      0.77       583
weighted avg       0.78      0.77      0.77       583

1
              precision    recall  f1-score   support

           0       0.87      0.66      0.75       145
           1       0.83      0.93      0.88       151
           2       0.67      0.76      0.71       142
           3       0.74      0.72      0.73       145

    accuracy                           0.77       583
   macro avg       0.78      0.77      0.77       583
weighted avg       0.78      0.77      0.77       583

2
              precision    recall  f1-score   support

           0       0.87      0.66      0.75       145
           1   

19
              precision    recall  f1-score   support

           0       0.87      0.66      0.75       145
           1       0.83      0.93      0.88       151
           2       0.67      0.76      0.71       142
           3       0.74      0.72      0.73       145

    accuracy                           0.77       583
   macro avg       0.78      0.77      0.77       583
weighted avg       0.78      0.77      0.77       583

20
              precision    recall  f1-score   support

           0       0.87      0.66      0.75       145
           1       0.83      0.93      0.88       151
           2       0.67      0.76      0.71       142
           3       0.74      0.72      0.73       145

    accuracy                           0.77       583
   macro avg       0.78      0.77      0.77       583
weighted avg       0.78      0.77      0.77       583

21
              precision    recall  f1-score   support

           0       0.87      0.66      0.75       145
           1

38
              precision    recall  f1-score   support

           0       0.87      0.66      0.75       145
           1       0.83      0.93      0.88       151
           2       0.67      0.76      0.71       142
           3       0.74      0.72      0.73       145

    accuracy                           0.77       583
   macro avg       0.78      0.77      0.77       583
weighted avg       0.78      0.77      0.77       583

39
              precision    recall  f1-score   support

           0       0.87      0.66      0.75       145
           1       0.83      0.93      0.88       151
           2       0.67      0.76      0.71       142
           3       0.74      0.72      0.73       145

    accuracy                           0.77       583
   macro avg       0.78      0.77      0.77       583
weighted avg       0.78      0.77      0.77       583

40
              precision    recall  f1-score   support

           0       0.87      0.66      0.75       145
           1

57
              precision    recall  f1-score   support

           0       0.87      0.66      0.75       145
           1       0.83      0.93      0.88       151
           2       0.67      0.76      0.71       142
           3       0.74      0.72      0.73       145

    accuracy                           0.77       583
   macro avg       0.78      0.77      0.77       583
weighted avg       0.78      0.77      0.77       583

58
              precision    recall  f1-score   support

           0       0.87      0.66      0.75       145
           1       0.83      0.93      0.88       151
           2       0.67      0.76      0.71       142
           3       0.74      0.72      0.73       145

    accuracy                           0.77       583
   macro avg       0.78      0.77      0.77       583
weighted avg       0.78      0.77      0.77       583

59
              precision    recall  f1-score   support

           0       0.87      0.66      0.75       145
           1

76
              precision    recall  f1-score   support

           0       0.87      0.66      0.75       145
           1       0.83      0.93      0.88       151
           2       0.67      0.76      0.71       142
           3       0.74      0.72      0.73       145

    accuracy                           0.77       583
   macro avg       0.78      0.77      0.77       583
weighted avg       0.78      0.77      0.77       583

77
              precision    recall  f1-score   support

           0       0.87      0.66      0.75       145
           1       0.83      0.93      0.88       151
           2       0.67      0.76      0.71       142
           3       0.74      0.72      0.73       145

    accuracy                           0.77       583
   macro avg       0.78      0.77      0.77       583
weighted avg       0.78      0.77      0.77       583

78
              precision    recall  f1-score   support

           0       0.87      0.66      0.75       145
           1

95
              precision    recall  f1-score   support

           0       0.87      0.66      0.75       145
           1       0.83      0.93      0.88       151
           2       0.67      0.76      0.71       142
           3       0.74      0.72      0.73       145

    accuracy                           0.77       583
   macro avg       0.78      0.77      0.77       583
weighted avg       0.78      0.77      0.77       583

96
              precision    recall  f1-score   support

           0       0.87      0.66      0.75       145
           1       0.83      0.93      0.88       151
           2       0.67      0.76      0.71       142
           3       0.74      0.72      0.73       145

    accuracy                           0.77       583
   macro avg       0.78      0.77      0.77       583
weighted avg       0.78      0.77      0.77       583

97
              precision    recall  f1-score   support

           0       0.87      0.66      0.75       145
           1

In [27]:
# test_sub=[10,11,13,14,15,16,17]
# for i in test_sub:
#     print(i)
#     train=df_new_1[df_new_1['subject']<=9]
#     test=df_new_1[df_new_1['subject']==i]

#     scaler = Normalizer()
#     scaled_data_train = scaler.fit_transform(train[sel_fea])
#     scaled_data_test = scaler.transform(test[sel_fea])
    
#     #et = ExtraTreesClassifier(n_estimators=100,n_jobs=10,random_state=61)
#     clf.fit(scaled_data_train,train['label'])
#     y_pred=clf.predict(scaled_data_test)
#     print(classification_report(test['label'],y_pred))

#GradientBoosting

In [17]:
#random state 3
scaler = Normalizer()
scaled_data_train = scaler.fit_transform(train[sel_fea])
scaled_data_test = scaler.transform(test[sel_fea])

from sklearn.ensemble import GradientBoostingClassifier
for i in range (101):
    clf= GradientBoostingClassifier(random_state=i)
    clf.fit(scaled_data_train,train['label'])
    y_pred=clf.predict(scaled_data_test)
#     print (classification_report(test['label'],y_pred))
    rpt = classification_report(test['label'],y_pred,output_dict=True)['accuracy']

    if rpt>=0.80:
        print (i)
        print (classification_report(test['label'],y_pred))

0
              precision    recall  f1-score   support

           0       0.88      0.66      0.75       145
           1       0.88      0.95      0.91       151
           2       0.70      0.80      0.75       142
           3       0.77      0.79      0.78       145

    accuracy                           0.80       583
   macro avg       0.81      0.80      0.80       583
weighted avg       0.81      0.80      0.80       583

3
              precision    recall  f1-score   support

           0       0.87      0.67      0.75       145
           1       0.88      0.95      0.92       151
           2       0.70      0.80      0.75       142
           3       0.77      0.77      0.77       145

    accuracy                           0.80       583
   macro avg       0.81      0.80      0.80       583
weighted avg       0.81      0.80      0.80       583

6
              precision    recall  f1-score   support

           0       0.89      0.66      0.76       145
           1   

56
              precision    recall  f1-score   support

           0       0.89      0.66      0.75       145
           1       0.88      0.95      0.91       151
           2       0.70      0.80      0.75       142
           3       0.77      0.79      0.78       145

    accuracy                           0.80       583
   macro avg       0.81      0.80      0.80       583
weighted avg       0.81      0.80      0.80       583

57
              precision    recall  f1-score   support

           0       0.89      0.66      0.75       145
           1       0.88      0.95      0.91       151
           2       0.70      0.80      0.75       142
           3       0.77      0.79      0.78       145

    accuracy                           0.80       583
   macro avg       0.81      0.80      0.80       583
weighted avg       0.81      0.80      0.80       583

58
              precision    recall  f1-score   support

           0       0.89      0.66      0.76       145
           1

In [ ]:
test_sub=[10,11,13,14,15,16,17]
for i in test_sub:
    print(i)
    train=df_new_1[df_new_1['subject']<=9]
    test=df_new_1[df_new_1['subject']==i]

    scaler = Normalizer()
    scaled_data_train = scaler.fit_transform(train[sel_fea])
    scaled_data_test = scaler.transform(test[sel_fea])
    
    #et = ExtraTreesClassifier(n_estimators=100,n_jobs=10,random_state=61)
    clf.fit(scaled_data_train,train['label'])
    y_pred=clf.predict(scaled_data_test)
    print(classification_report(test['label'],y_pred))

#SVM-linear

In [ ]:
#scaler = Normalizer()
#scaled_data_train = scaler.fit_transform(train[sel_fea])
#scaled_data_test = scaler.transform(test[sel_fea])

from sklearn import svm
clf = svm.SVC(kernel='linear')
clf.fit(train[sel_fea],train['label'])
y_pred=clf.predict(test[sel_fea])
print (classification_report(test['label'],y_pred))

In [ ]:
test_sub=[10,11,13,14,15,16,17]
for i in test_sub:
    print(i)
    train=df_new_1[df_new_1['subject']<=9]
    test=df_new_1[df_new_1['subject']==i]

    #scaler = Normalizer()
    #scaled_data_train = scaler.fit_transform(train[sel_fea])
    #scaled_data_test = scaler.transform(test[sel_fea])
    
    #et = ExtraTreesClassifier(n_estimators=100,n_jobs=10,random_state=61)
    clf.fit(train[sel_fea],train['label'])
    y_pred=clf.predict(test[sel_fea])
    print(classification_report(test['label'],y_pred))

#SVM-poly

In [ ]:
from sklearn import svm
clf = svm.SVC(kernel='poly')
clf.fit(train[sel_fea],train['label'])
y_pred=clf.predict(test[sel_fea])
print (classification_report(test['label'],y_pred))

In [ ]:
test_sub=[10,11,13,14,15,16,17]
for i in test_sub:
    print(i)
    train=df_new_1[df_new_1['subject']<=9]
    test=df_new_1[df_new_1['subject']==i]

    #scaler = Normalizer()
    #scaled_data_train = scaler.fit_transform(train[sel_fea])
    #scaled_data_test = scaler.transform(test[sel_fea])
    
    #et = ExtraTreesClassifier(n_estimators=100,n_jobs=10,random_state=61)
    clf.fit(train[sel_fea],train['label'])
    y_pred=clf.predict(test[sel_fea])
    print(classification_report(test['label'],y_pred))

#SVM -RBF

In [ ]:
from sklearn import svm
clf = svm.SVC(kernel='rbf')
clf.fit(train[sel_fea],train['label'])
y_pred=clf.predict(test[sel_fea])
print (classification_report(test['label'],y_pred))

In [ ]:
test_sub=[10,11,13,14,15,16,17]
for i in test_sub:
    print(i)
    train=df_new_1[df_new_1['subject']<=9]
    test=df_new_1[df_new_1['subject']==i]

    #scaler = Normalizer()
    #scaled_data_train = scaler.fit_transform(train[sel_fea])
    #scaled_data_test = scaler.transform(test[sel_fea])
    
    #et = ExtraTreesClassifier(n_estimators=100,n_jobs=10,random_state=61)
    clf.fit(train[sel_fea],train['label'])
    y_pred=clf.predict(test[sel_fea])
    print(classification_report(test['label'],y_pred))

#ADABOOST

In [38]:
scaler = Normalizer()
scaled_data_train = scaler.fit_transform(train[sel_fea])
scaled_data_test = scaler.transform(test[sel_fea])

from sklearn.ensemble import AdaBoostClassifier
for i in range (101):
    clf = AdaBoostClassifier(n_estimators=50,learning_rate=0.01, random_state=i)
    clf.fit(scaled_data_train,train['label'])
    y_pred=clf.predict(scaled_data_test)
#     print (classification_report(test['label'],y_pred))
    
    rpt = classification_report(test['label'],y_pred,output_dict=True)['accuracy']
    
    if rpt >= 0.55:
        print (i)
        print (classification_report(test['label'],y_pred))

0
              precision    recall  f1-score   support

           0       0.45      0.65      0.54       147
           1       0.93      0.60      0.73       161
           2       0.48      0.60      0.53       146
           3       0.65      0.48      0.55       149

    accuracy                           0.58       603
   macro avg       0.63      0.58      0.59       603
weighted avg       0.64      0.58      0.59       603

1
              precision    recall  f1-score   support

           0       0.45      0.65      0.54       147
           1       0.93      0.60      0.73       161
           2       0.48      0.60      0.53       146
           3       0.65      0.48      0.55       149

    accuracy                           0.58       603
   macro avg       0.63      0.58      0.59       603
weighted avg       0.64      0.58      0.59       603

2
              precision    recall  f1-score   support

           0       0.45      0.65      0.54       147
           1   

20
              precision    recall  f1-score   support

           0       0.45      0.65      0.54       147
           1       0.93      0.60      0.73       161
           2       0.48      0.60      0.53       146
           3       0.65      0.48      0.55       149

    accuracy                           0.58       603
   macro avg       0.63      0.58      0.59       603
weighted avg       0.64      0.58      0.59       603

21
              precision    recall  f1-score   support

           0       0.45      0.65      0.54       147
           1       0.93      0.60      0.73       161
           2       0.48      0.60      0.53       146
           3       0.65      0.48      0.55       149

    accuracy                           0.58       603
   macro avg       0.63      0.58      0.59       603
weighted avg       0.64      0.58      0.59       603

22
              precision    recall  f1-score   support

           0       0.45      0.65      0.54       147
           1

40
              precision    recall  f1-score   support

           0       0.45      0.65      0.54       147
           1       0.93      0.60      0.73       161
           2       0.48      0.60      0.53       146
           3       0.65      0.48      0.55       149

    accuracy                           0.58       603
   macro avg       0.63      0.58      0.59       603
weighted avg       0.64      0.58      0.59       603

41
              precision    recall  f1-score   support

           0       0.45      0.65      0.54       147
           1       0.93      0.60      0.73       161
           2       0.48      0.60      0.53       146
           3       0.65      0.48      0.55       149

    accuracy                           0.58       603
   macro avg       0.63      0.58      0.59       603
weighted avg       0.64      0.58      0.59       603

42
              precision    recall  f1-score   support

           0       0.45      0.65      0.54       147
           1

60
              precision    recall  f1-score   support

           0       0.45      0.65      0.54       147
           1       0.93      0.60      0.73       161
           2       0.48      0.60      0.53       146
           3       0.65      0.48      0.55       149

    accuracy                           0.58       603
   macro avg       0.63      0.58      0.59       603
weighted avg       0.64      0.58      0.59       603

61
              precision    recall  f1-score   support

           0       0.45      0.65      0.54       147
           1       0.93      0.60      0.73       161
           2       0.48      0.60      0.53       146
           3       0.65      0.48      0.55       149

    accuracy                           0.58       603
   macro avg       0.63      0.58      0.59       603
weighted avg       0.64      0.58      0.59       603

62
              precision    recall  f1-score   support

           0       0.45      0.65      0.54       147
           1

80
              precision    recall  f1-score   support

           0       0.45      0.65      0.54       147
           1       0.93      0.60      0.73       161
           2       0.48      0.60      0.53       146
           3       0.65      0.48      0.55       149

    accuracy                           0.58       603
   macro avg       0.63      0.58      0.59       603
weighted avg       0.64      0.58      0.59       603

81
              precision    recall  f1-score   support

           0       0.45      0.65      0.54       147
           1       0.93      0.60      0.73       161
           2       0.48      0.60      0.53       146
           3       0.65      0.48      0.55       149

    accuracy                           0.58       603
   macro avg       0.63      0.58      0.59       603
weighted avg       0.64      0.58      0.59       603

82
              precision    recall  f1-score   support

           0       0.45      0.65      0.54       147
           1

100
              precision    recall  f1-score   support

           0       0.45      0.65      0.54       147
           1       0.93      0.60      0.73       161
           2       0.48      0.60      0.53       146
           3       0.65      0.48      0.55       149

    accuracy                           0.58       603
   macro avg       0.63      0.58      0.59       603
weighted avg       0.64      0.58      0.59       603



In [ ]:
test_sub=[10,11,13,14,15,16,17]
for i in test_sub:
    print(i)
    train=df_new_1[df_new_1['subject']<=9]
    test=df_new_1[df_new_1['subject']==i]

    scaler = Normalizer()
    scaled_data_train = scaler.fit_transform(train[sel_fea])
    scaled_data_test = scaler.transform(test[sel_fea])
    
    #et = ExtraTreesClassifier(n_estimators=100,n_jobs=10,random_state=61)
    clf.fit(scaled_data_train,train['label'])
    y_pred=clf.predict(scaled_data_test)
    print(classification_report(test['label'],y_pred))